In [1]:
import numpy as np 
import pandas as pd 
import os, gc, re, warnings
warnings.filterwarnings("ignore")

In [2]:
dftr = pd.read_csv("/home/gunjan/Desktop/amazon/datasetb2d9982/dataset/train.csv")
dftr["src"]="train"
dfte = pd.read_csv("/home/gunjan/Desktop/amazon/datasetb2d9982/dataset/test.csv")
dfte["src"]="test"
print('Train shape:',dftr.shape,'Test shape:',dfte.shape,'Test columns:',dfte.columns)
# df = pd.concat([dftr,dfte],ignore_index=True)

dftr.head()

Train shape: (2249698, 7) Test shape: (734736, 6) Test columns: Index(['PRODUCT_ID', 'TITLE', 'BULLET_POINTS', 'DESCRIPTION',
       'PRODUCT_TYPE_ID', 'src'],
      dtype='object')


,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH,src
0,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,NaN,1650,2125.980000,train
1,2673191,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...,"[Harry Potter Hedwig Pyjamas (6-16 Yrs),100% c...",NaN,2755,393.700000,train
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537,748.031495,train
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996,787.401574,train
4,283658,The United Empire Loyalists: A Chronicle of th...,NaN,NaN,6112,598.424000,train


In [3]:
print(dftr.shape)
dftr = dftr.dropna(axis=0, subset=['TITLE'])
print(dftr.shape)

(2249698, 7)
(2249686, 7)


In [4]:
dftr.fillna("", inplace=True)
dfte.fillna("", inplace=True)

In [5]:
dftr.isna().sum()

PRODUCT_ID         0
TITLE              0
BULLET_POINTS      0
DESCRIPTION        0
PRODUCT_TYPE_ID    0
PRODUCT_LENGTH     0
src                0
dtype: int64

In [6]:
dfte.isna().sum()

PRODUCT_ID         0
TITLE              0
BULLET_POINTS      0
DESCRIPTION        0
PRODUCT_TYPE_ID    0
src                0
dtype: int64

In [7]:
dfte.shape

(734736, 6)

In [8]:
target_cols = ['PRODUCT_LENGTH']

In [ ]:
# import sys
# sys.path.append('../input/iterativestratification')
# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
# FOLDS = 25
# skf = MultilabelStratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)
# for i,(train_index, val_index) in enumerate(skf.split(dftr,dftr[target_cols])):
#     dftr.loc[val_index,'FOLD'] = i
# print('Train samples per fold:')
# dftr.FOLD.value_counts()

In [4]:
dftr.iloc[0]

PRODUCT_ID                                                   1925202
TITLE              ArtzFolio Tulip Flowers Blackout Curtain for D...
BULLET_POINTS      [LUXURIOUS & APPEALING: Beautiful custom-made ...
DESCRIPTION                                                      NaN
PRODUCT_TYPE_ID                                                 1650
PRODUCT_LENGTH                                               2125.98
src                                                            train
Name: 0, dtype: object

In [7]:
from sklearn.model_selection import StratifiedKFold
def create_folds(data, num_splits):
    data["kfold"] = -1
    data = data.sample(frac=1).reset_index(drop=True)
    y=data["PRODUCT_TYPE_ID"]
    kf = StratifiedKFold(n_splits=num_splits)
    for f, (t_, v_) in enumerate(kf.split(X=data, y=y)):
        data.loc[v_, 'kfold'] = f
    return data

In [ ]:
train=create_folds(dftr, 5)

In [ ]:
dftr=train.loc[train.kfold.isin([0])]
dftr=dftr.reset_index(drop=True)
display(dftr.head())
dftr.shape

In [ ]:
del train
gc.collect()

# Generate Embeddings

In [9]:
from transformers import AutoModel,AutoTokenizer, DataCollatorWithPadding
import torch
import torch.nn.functional as F
from tqdm import tqdm

In [10]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state.detach().cpu()
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(
        input_mask_expanded.sum(1), min=1e-9
    )

In [11]:
BATCH_SIZE = 32
tokenizer = None
MAX_LEN = 32

class EmbedDataset(torch.utils.data.Dataset):
    def __init__(self,df):
        self.df = df.reset_index(drop=True)
    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        text = self.df.loc[idx,"TITLE"]
        tokens = tokenizer(
                text,
                None,
                add_special_tokens=True,
                padding=False,
                truncation=True,
                max_length=32,return_tensors="pt")
        tokens = {k:v.squeeze(0) for k,v in tokens.items()}
        return tokens



In [12]:
def get_embeddings(MODEL_NM='', MAX=640, BATCH_SIZE=4, verbose=True):
    global tokenizer, MAX_LEN
    DEVICE="cuda"
    model = AutoModel.from_pretrained( MODEL_NM )
    tokenizer = AutoTokenizer.from_pretrained( MODEL_NM )
    MAX_LEN = MAX
    
    ds_tr = EmbedDataset(dftr)
    embed_dataloader_tr = torch.utils.data.DataLoader(ds_tr,\
                        batch_size=BATCH_SIZE,\
                        shuffle=False, collate_fn=DataCollatorWithPadding(tokenizer))
    ds_te = EmbedDataset(dfte)
    embed_dataloader_te = torch.utils.data.DataLoader(ds_te,\
                        batch_size=BATCH_SIZE,\
                        shuffle=False, collate_fn=DataCollatorWithPadding(tokenizer))
    
    model = model.to(DEVICE)
    model.eval()
    all_train_text_feats = []
    for batch in tqdm(embed_dataloader_tr,total=len(embed_dataloader_tr)):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        with torch.no_grad():
            model_output = model(input_ids=input_ids,attention_mask=attention_mask)
        sentence_embeddings = mean_pooling(model_output, attention_mask.detach().cpu())
        # Normalize the embeddings
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        sentence_embeddings =  sentence_embeddings.squeeze(0).detach().cpu().numpy()
        all_train_text_feats.extend(sentence_embeddings)
    all_train_text_feats = np.array(all_train_text_feats)
    if verbose:
        print('Train embeddings shape',all_train_text_feats.shape)
        
    te_text_feats = []
    for batch in tqdm(embed_dataloader_te,total=len(embed_dataloader_te)):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        with torch.no_grad():
            model_output = model(input_ids=input_ids,attention_mask=attention_mask)
        sentence_embeddings = mean_pooling(model_output, attention_mask.detach().cpu())
        # Normalize the embeddings
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        sentence_embeddings =  sentence_embeddings.squeeze(0).detach().cpu().numpy()
        te_text_feats.extend(sentence_embeddings)
    te_text_feats = np.array(te_text_feats)
    if verbose:
        print('Test embeddings shape',te_text_feats.shape)
        
    return all_train_text_feats, te_text_feats

In [13]:
MODEL_NM = '../input/huggingface-deberta-variants/deberta-base/deberta-base'
all_train_text_feats, te_text_feats = get_embeddings(MODEL_NM, MAX=32, BATCH_SIZE=32)

Some weights of the model checkpoint at ../input/huggingface-deberta-variants/deberta-base/deberta-base were not used when initializing DebertaModel: ['config', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.bias']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
 29%|██▉       | 20426/70303 [17:41<42:36, 19.51it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To c

KeyboardInterrupt: 

In [ ]:
np.save('train_deberta_base_feats.npy', all_train_text_feats)
np.save('test_deberta_base_feats.npy', te_text_feats)

In [ ]:
del all_train_text_feats, te_text_feats
gc.collect()

In [ ]:
MODEL_NM = '../input/deberta-v3-large/deberta-v3-large'
all_train_text_feats2, te_text_feats2 = get_embeddings(MODEL_NM, MAX=32, BATCH_SIZE=32)

In [ ]:
np.save('train_deberta_largev3_feats.npy', all_train_text_feats2)
np.save('test_deberta_largev3_feats.npy', te_text_feats2)

In [ ]:
del all_train_text_feats2, te_text_feats2
gc.collect()